## Appendix B
# Elementary Audio Digital Filters

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import sys
sys.path.append('../')
from ipython_animation import create_animation, DEFAULT_FPS
from plot_utils import zplane

In [3]:
from scipy.signal import freqz, lfilter

def animate_freq_response_with_pz(parameters_for_frame,
                                  animation_length_seconds=4,
                                  figsize=(12, 6.75),
                                  amp_response_ylim=(0, 2),
                                  phase_response_ylim=(-np.pi/2, np.pi/2),
                                  amp_response_yticks=None,
                                  phase_response_yticks=None,
                                  show_impulse_response=True):
    global zeros_ax, poles_ax
    zeros_ax, poles_ax = None, None

    num_frames = animation_length_seconds * DEFAULT_FPS
    w, H = freqz([1], [1]) # get a default frequency axis

    fig = plt.figure(figsize=figsize)
    num_rows = 3 if show_impulse_response else 2
    amplitude_axis = plt.subplot2grid((num_rows, 4), (0, 0), colspan=2)
    phase_axis = plt.subplot2grid((num_rows, 4), (1, 0), colspan=2)
    if show_impulse_response:
        ir_axis = plt.subplot2grid((num_rows, 4), (2, 0), colspan=2)
    zp_axis = plt.subplot2grid((num_rows, 4), (0, 2), rowspan=num_rows, colspan=2)

    amplitude_line, = amplitude_axis.plot(w, np.zeros(w.size), c='red', linewidth=3)
    amplitude_axis.set_title('Amplitude response', size=13)
    amplitude_axis.set_xlabel('Normalized Frequency (rad/sample)')
    amplitude_axis.set_ylabel('Gain $\\longrightarrow$')
    amplitude_axis.grid(True)
    amplitude_axis.set_xlim(w[0], w[-1])    
    amplitude_axis.set_ylim(amp_response_ylim)
    if amp_response_yticks is not None:
        amplitude_axis.set_yticks(amp_response_yticks)

    phase_line, = phase_axis.plot(w, np.zeros(w.size), c='green', linewidth=3)
    phase_axis.set_title('Phase response', size=13)
    phase_axis.set_xlabel('Normalized Frequency (rad/sample)')
    phase_axis.set_ylabel('Phase Shift (rad)')
    phase_axis.grid(True)
    phase_axis.set_xlim(w[0], w[-1])
    phase_axis.set_ylim(phase_response_ylim)
    if phase_response_yticks is not None:
        phase_axis.set_yticks(phase_response_yticks)

    if show_impulse_response:
        impulse = [1.0] + [0] * 99
        ir_line, = ir_axis.plot(impulse, c='black', linewidth=3)
        ir_axis.set_title('Impulse response', size=13)
        ir_axis.set_xlabel('Time (samples)')
        ir_axis.set_ylabel('Amplitude $\\longrightarrow$')
        ir_axis.grid(True)
        ir_axis.set_xlim(0, len(impulse))
        ir_axis.set_ylim(-1, 1)

    def animate(frame):
        global zeros_ax, poles_ax

        parameters = parameters_for_frame(frame, num_frames)
        fig.suptitle(parameters['title'], size=14)

        B = parameters['B']; A = parameters['A']
        w, H = freqz(B, A)
        amplitude_line.set_ydata(np.abs(H))
        phase_line.set_ydata(np.angle(H))

        if show_impulse_response:
            h = lfilter(B, A, impulse)
            ir_line.set_ydata(h)

        zeros_ax, poles_ax = zplane(parameters['zeros'], parameters['poles'], zp_axis, zeros_ax, poles_ax)
        
        plt.tight_layout()
        plt.subplots_adjust(top=(0.9 - (parameters['title'].count('\n') * 0.05)))

    return create_animation(fig, plt, animate, length_seconds=animation_length_seconds)

## Elementary Filter Sections

### One-Zero

![](https://ccrma.stanford.edu/~jos/filters/img1342.png)
![](https://ccrma.stanford.edu/~jos/filters/img1343.png)
![](https://ccrma.stanford.edu/~jos/filters/img1347.png)

The following animation is inspired by [Figure B.2](https://ccrma.stanford.edu/~jos/filters/One_Zero.html) in the book:

![](https://ccrma.stanford.edu/~jos/filters/img1346.png)

In [4]:
def parameters_for_frame(frame, num_frames):
    percent_complete = frame / (num_frames - 1)
    b_0 = 1
    # range [-1, 1]
    if percent_complete <= 0.5:
        b_1 = 4 * percent_complete - 1
    else:
        b_1 = 1 + 4 * (0.5 - percent_complete)

    return {'B': [b_0, b_1], 'A': [1], 'title': 'One-zero filter\n$y(n) = x(n) + b_1x(n-1)$, with $b_1=%0.3f$' % b_1, 'zeros': [-b_1/b_0], 'poles': []}
    
animate_freq_response_with_pz(parameters_for_frame, amp_response_yticks=[0, 0.5, 1, 1.5, 2])

### One-Pole

![](https://ccrma.stanford.edu/~jos/filters/img1352.png)
![](https://ccrma.stanford.edu/~jos/filters/img1354.png)
![](https://ccrma.stanford.edu/~jos/filters/img1355.png)

The following animation is inspired by [Figure B.4](https://ccrma.stanford.edu/~jos/filters/One_Pole.html) in the book:

![](https://ccrma.stanford.edu/~jos/filters/img1353.png)

In [5]:
def parameters_for_frame(frame, num_frames):
    percent_complete = frame / (num_frames - 1)
    # range [-0.9, 0.9]
    if percent_complete <= 0.5:
        a_1 = 3.6 * percent_complete - 0.9
    else:
        a_1 = 0.92 + 3.6 * (0.5 - percent_complete)
    return {'B': [1], 'A': [1, a_1], 'title': 'One-pole filter\n$y(n) = x(n) - a_1y(n-1)$, with $a_1=%0.3f$' % a_1, 'zeros': [0], 'poles': [-a_1]}
    
animate_freq_response_with_pz(parameters_for_frame, amp_response_ylim=(0, 10))

### Two-Pole

![](https://ccrma.stanford.edu/~jos/filters/img1359.png)
![](https://ccrma.stanford.edu/~jos/filters/img1360.png)
![](https://ccrma.stanford.edu/~jos/filters/img1389.png)

The following animation is inspired by [Figure B.6](https://ccrma.stanford.edu/~jos/filters/Resonator_Bandwidth_Terms_Pole.html) in the book:

![](https://ccrma.stanford.edu/~jos/filters/img1392.png)

In [6]:
def parameters_for_frame(frame, num_frames):
    percent_complete = frame / (num_frames - 1)
    if percent_complete <= 0.25:
        R = 0.25 + 3 * percent_complete
    else:
        R = 1.25 - percent_complete
    theta_c = 2 * np.pi * percent_complete # poles will spiral out
    a_1 = -2 * R * np.cos(theta_c); a_2 = R ** 2

    return {'B': [1], 'A': [1, a_1, a_2], 'title': 'Two-pole filter\n$y(n) = x(n) - a_1y(n-1) - a_2y(n-2)$,\nwith $a_1=-2R\cos(\\theta_c)$, $a_2=R^2$, $R=%0.3f, \\theta_c=%0.3f$' % (R, theta_c), 'zeros': [0], 'poles': [R * np.exp(1j * theta_c), R * np.exp(-1j * theta_c)]}
    
animate_freq_response_with_pz(parameters_for_frame, amp_response_ylim=(0, 10), phase_response_ylim=(-np.pi / 2, np.pi))

### Two-Zero

![](https://ccrma.stanford.edu/~jos/filters/img1394.png)
![](https://ccrma.stanford.edu/~jos/filters/img1393.png)
![](https://ccrma.stanford.edu/~jos/filters/img1389.png)

The following animation is based off of [Figure B.8](https://ccrma.stanford.edu/~jos/filters/Two_Zero.html) in the book:

![](https://ccrma.stanford.edu/~jos/filters/img1405.png)

In [7]:
def parameters_for_frame(frame, num_frames):
    percent_complete = frame / (num_frames - 1)
    if percent_complete <= 0.25:
        R = 0.25 + 3 * percent_complete
    else:
        R = 1.25 - percent_complete
    theta_c = 2 * np.pi * percent_complete # poles will spiral out
    b_1 = -2 * R * np.cos(theta_c); b_2 = R ** 2

    return {'B': [1, b_1, b_2], 'A': [1], 'title': 'Two-zero filter\n$y(n) = x(n) + b_1x(n-1) + b_2x(n-2)$,\nwith $b_1=-2R\cos(\\theta_c)$, $b_2=R^2$, $R=%0.3f, \\theta_c=%0.3f$' % (R, theta_c), 'zeros': [R * np.exp(1j * theta_c), R * np.exp(-1j * theta_c)], 'poles': [0]}
    
animate_freq_response_with_pz(parameters_for_frame, amp_response_ylim=(0, 4), phase_response_ylim=(-np.pi, np.pi/2))

### Complex One-Pole Resonator

In [8]:
def parameters_for_frame(frame, num_frames):
    percent_complete = frame / (num_frames - 1)
    if percent_complete <= 0.25:
        g = 0.25 + 3 * percent_complete
    else:
        g = 1.25 - percent_complete
    omega_c = 2 * np.pi * percent_complete # poles will spiral out
    p = 0.99 * np.exp(1j * omega_c)

    return {'B': [g], 'A': [1, -p], 'title': 'Complex one-pole resonator\n$y(n) = gx(n) + py(n-1)$,\nwith $p=e^{j\\omega_c}$, $g=%0.3f, \\omega_c=%0.3f$' % (g, omega_c), 'zeros': [0], 'poles': [p]}
    
animate_freq_response_with_pz(parameters_for_frame, amp_response_ylim=(0, 16), phase_response_ylim=(-np.pi / 2, np.pi))

/usr/local/lib/python3.6/site-packages/numpy/core/numeric.py:531: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


### Biquad Section

_The following section summarizes_ [section B.1.6](https://ccrma.stanford.edu/~jos/filters/BiQuad_Section.html):

A biquad is a common name for a two-pole, two-zero digital filter. The transfer function of the biquad can be defined as

$H(z) = g\frac{1 + \beta_1z^{-1}+\beta_2z^{-2}}{1 + a_1z^{-1} + a_2z^{-2}},$

where $g$ can be called the _overall gain_ of the biquad.

Just like in the two-pole case above, we can express the denominator coefficients in terms of the radius $R$ and angle $\theta$ of the positive frequency pole. We can think of $\theta$ as the _resonance frequency_ (in radians per sample $\theta = 2\pi f_cT$, where $f_c$ is the resonance frequency in Hz), and $R$ as determining the "Q" of the resonance.

If the numerator is expressed this way, we can think of the zero-angle as the _antiresonance frequency_, and the zero-radius affects the _depth_ and _width_ of the antiresonance (or _notch_).

A common setting for the zeros for a resonator is to place one at $z=1$ (dc) and the other at $z=-1$ (half the sampling rate). This zero placement normalizes the peak gain of the resonator if it is swept using the $a_1$ parameter.

In [9]:
def parameters_for_frame(frame, num_frames):
    g = 1; beta_1 = 0; beta_2 = -1 # corresponds to zeros at -1 and 1 for peak normalization
    percent_complete = frame / (num_frames - 1)
    if percent_complete <= 0.25:
        R = 0.25 + 3 * percent_complete
    else:
        R = 1.25 - percent_complete
    theta_c = 2 * np.pi * percent_complete # poles will spiral out
    a_1 = -2 * R * np.cos(theta_c); a_2 = R ** 2

    return {'B': [g, g * beta_1, g * beta_2], 'A': [1, a_1, a_2], 'title': 'Biquad Section\n$y(n) = gx(n) + g\\beta_1 x(n - 1) + g\\beta_2 x(n-2) - a_1y(n-1) - a_2y(n-2)$,\nwith $g=%0.2f, \\beta_1 = %0.2f, \\beta_2 = %0.2f, a_1=-2R\cos(\\theta_c), a_2=R^2, R=%0.3f, \\theta_c=%0.3f$' % (g, beta_1, beta_2, R, theta_c), 'zeros': [-1, 1], 'poles': [R * np.exp(1j * theta_c), R * np.exp(-1j * theta_c)]}
    
animate_freq_response_with_pz(parameters_for_frame, amp_response_ylim=(0, 16), phase_response_ylim=(-np.pi / 2, np.pi))

### Biquad Allpass Filter Section

An allpass filter must satisfy

$\left|H(e^{j\omega T})\right| = 1$.

For the biquad case, we can fullfill this critera by setting $B(z) = z^{-2}A(z^{-1}) = a_2 + a_1z^{-1} + z^{-2}$.

Thus, the numerator polynomial is simply the "flip" of the denominator polynomial. To obtain unity gain, we set $g = a_2, \beta_1 = a_1/a_2$, and $\beta_2 = 1/a_2$.

In [10]:
def parameters_for_frame(frame, num_frames):
    percent_complete = frame / (num_frames - 1)
    if percent_complete <= 0.5:
        R = 0.75 + 0.5 * percent_complete
    else:
        R = 1.25 - 0.5 * percent_complete
    theta_c = 2 * np.pi * percent_complete # poles will spiral out
    a_1 = -2 * R * np.cos(theta_c); a_2 = R ** 2

    poles = np.array([R * np.exp(1j * theta_c), R * np.exp(-1j * theta_c)])

    return {'B': [np.conj(a_2), np.conj(a_1), 1], 'A': [1, a_1, a_2], 'title': 'Biquad Allpass Section\n$y(n) = a_2x(n) + a_1x(n - 1) + x(n-2) - a_1y(n-1) - a_2y(n-2)$,\nwith $a_1=-2R\cos(\\theta_c), a_2=R^2, R=%0.3f, \\theta_c=%0.3f$' % (R, theta_c), 'zeros': 1 / poles, 'poles': poles}
    
animate_freq_response_with_pz(parameters_for_frame, amp_response_ylim=(0, 2), phase_response_ylim=(-np.pi, np.pi))

### DC Blocker

Transfer function:

$H(z) = \frac{1 - z^{-1}}{1 - Rz^{-1}}$

There is a zero at dc ($z = 1$) and a pole near dc at $z = R$.

![](https://ccrma.stanford.edu/~jos/filters/img1462.png)

Note that we can scale the impulse response by the inverse of the maximum gain to bound the amplitude response to 1 for all frequencies.

In [11]:
def parameters_for_frame(frame, num_frames):
    percent_complete = frame / (num_frames - 1)
    if percent_complete <= 0.5:
        R = 2 * percent_complete
    else:
        R = 2 - 2 * percent_complete

    return {'B': [1, -1], 'A': [1, -R], 'title': 'DC Blocker\n$y(n) = x(n) - x(n - 1) + Ry(n-1)$, with $R=%0.3f$' % R, 'zeros': [1], 'poles': [R]}
    
animate_freq_response_with_pz(parameters_for_frame, amp_response_ylim=(0, 2), phase_response_ylim=(0, np.pi/2))

## Elementary Filter Problems

TODO